In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel,BertForMaskedLM
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
import torch
from transformers import BertForMaskedLM, BertTokenizer
from tqdm import tqdm


/home/kd/anaconda3/envs/csci544/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labeled_comments = pd.read_csv('../data/pos_tags_dataset.csv')
input_sentences = labeled_comments['cleaned_comments'].tolist()
input_sentences = [sentence for sentence in input_sentences if type(sentence)==str and  len(sentence.split()) > 2]
with open('../data/bad_word.txt', 'r') as f:
    not_predict = f.read().split('\n')

In [3]:
with open('../data/bad_word.txt', 'r') as f:
    not_predict = f.read().split('\n')

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = model.to(device)
avoid_list = not_predict
avoid_list_set = set(avoid_list)
def preprocess(text, avoid_list_set):
    indices = []
    text_tokens = tokenizer.tokenize(text)
    for i, token in enumerate(text_tokens):
        if text_tokens[i].lower() in avoid_list_set:
            text_tokens[i] = '[AVOID]'
            indices.append(i)
    return ' '.join(text_tokens), indices


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import torch.nn as nn

class MaskedLMWithAvoidLoss(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        avoid_mask = (input_ids == tokenizer.convert_tokens_to_ids('[AVOID]'))
        avoid_mask = avoid_mask.logical_not()
        loss = self.model(input_ids, attention_mask, token_type_ids, labels=labels)[0]
        loss = (loss * avoid_mask.float()).mean()
        return loss

loss_function = MaskedLMWithAvoidLoss(model)


In [6]:
from transformers import AdamW
from torch.utils.data import DataLoader

# train_data = ["I like to eat pizza and spaghetti",
#               "I don't like to eat sushi",
#               "I want to eat something new today"]
train_data = input_sentences
train_data_processed = [preprocess(text, avoid_list)[0] for text in train_data]
train_data_tokenized = tokenizer(train_data_processed, return_tensors='pt', padding=True,truncation=True,max_length=30)
input_ids,token_type_ids,attention_mask = train_data_tokenized['input_ids'],train_data_tokenized['token_type_ids'],train_data_tokenized['attention_mask']
train_data = torch.utils.data.TensorDataset(input_ids,token_type_ids,attention_mask)
train_loader = DataLoader( train_data, batch_size=32, shuffle=True)


valid_data = ["I like to eat pussy",
        "I don't like to eat dick",
        "I want to eat ass for dinner"]
valid_data_processed = [preprocess(text, avoid_list)[0] for text in valid_data]
valid_data_tokenized = tokenizer(valid_data_processed, padding=True, truncation=True, return_tensors='pt',max_length=30)
input_ids,token_type_ids,attention_mask = valid_data_tokenized['input_ids'],valid_data_tokenized['token_type_ids'],valid_data_tokenized['attention_mask']
valid_data = torch.utils.data.TensorDataset(input_ids,token_type_ids,attention_mask)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

In [7]:
print(f'Using device {device}')
num_epochs = 10
optimizer = AdamW(loss_function.model.parameters(), lr=5e-5)
for epoch in range(num_epochs):
    total_loss = 0
    train_loss = 0
    loss_function.model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        token_type_ids = batch[2].to(device)
        labels = input_ids.clone()
        labels[labels == tokenizer.mask_token_id] = -100
        loss = loss_function(input_ids, attention_mask, token_type_ids, labels)
        loss.backward()
        train_loss += loss.item()
        # print(f"Training loss: {loss.item():.4f}")
        optimizer.step()
        loss_function.model.eval()

    # for batch in valid_loader:
    #     with torch.no_grad():
    #         input_ids = batch[0].to(device)
    #         attention_mask = batch[1].to(device)
    #         token_type_ids = batch[2].to(device)
    #         labels = input_ids.clone()
    #         labels[labels == tokenizer.mask_token_id] = -100
    #         loss = loss_function(input_ids, attention_mask, token_type_ids, labels)
    #         total_loss += loss.item() * len(batch)
    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1} completed. Train loss: {train_loss:.4f}")
    #Save model
    torch.save(loss_function.model.state_dict(), f'models/finetuned_bert_epoch_{epoch}.model')

/home/kd/anaconda3/envs/csci544/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using device cuda


100%|██████████| 1957/1957 [36:03<00:00,  1.11s/it]


Epoch 1 completed. Train loss: 4.2373


100%|██████████| 1957/1957 [36:23<00:00,  1.12s/it]


Epoch 2 completed. Train loss: 4.6206


100%|██████████| 1957/1957 [36:04<00:00,  1.11s/it]


Epoch 3 completed. Train loss: 4.9205


 76%|███████▌  | 1487/1957 [27:33<08:42,  1.11s/it]


KeyboardInterrupt: 

TODO: Fix for multiple MASK tokens

In [ ]:
#Example sentence
text = "I like to eat pussy and ass"
text="shit i have been sitting pretty nicely with a decent six figure income decent chance i am having a income shortly anyone need a good embedded firmware software test lead"
text="wut they are right the kardashians are full of shit xd your pussy do not need no gummy"
processed_text,indices = preprocess(text, avoid_list)
print(processed_text)
#Output: I like to eat [AVOID] and spaghetti

#Predict
tokenized_text = tokenizer(processed_text, return_tensors='pt')
input_ids = tokenized_text['input_ids'].to(device)
attention_mask = tokenized_text['attention_mask'].to(device)
token_type_ids = tokenized_text['token_type_ids'].to(device)
with torch.no_grad():
    output = loss_function.model(input_ids, attention_mask, token_type_ids)
    predictions = output[0]
    top_10_predicted_indices = torch.argmax(predictions[0][:10],dim=1)
    predicted_tokens = [tokenizer.convert_ids_to_tokens([index])[0] for index in top_10_predicted_indices]

# print(f"Predicted sentences: {[processed_text.replace('[AVOID]', predicted_token) for predicted_token in predicted_tokens]}")
print(predicted_tokens)

wu ##t they are right the ka ##rda ##shi ##ans are full of [AVOID] x ##d your [AVOID] do not need no gum ##my
['the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']
